## <font color=darkblue>  American Options pricing using Binomial tree </font>

## <font color=green> American Call option <font color=darkblue>

* CMP = 95
* K = 90
* T = 1 year
* r = 0 
* 5 steps

In [2]:
call, put, european, american = 100, 101, 102, 103

Setting the upper movement of the price (u): <br>

u = 1.30 <br>
d = 1/u 

In [3]:
side = call  # Option side
style = american  # Option style
price = 95  #Current instrument price
strike = 90  #Strike price
riskfree = 0  
divyield = 0
tte = 365  # Time to expiration in days

print('Calculation Inputs')
print('%18s : %0.3f' % ('Price', price))
print('%18s : %0.3f' % ('Strike', strike))
print('%18s : %0.3f' % ('Risk-free', riskfree))
print('%18s : %0.3f' % ('Div Yield', divyield))
print('%18s : %0.3f' % ('TTE Days', tte))
print()

Calculation Inputs
             Price : 95.000
            Strike : 90.000
         Risk-free : 0.000
         Div Yield : 0.000
          TTE Days : 365.000



In [4]:
n = 5  # Depth of binomial tree 
tdelta = tte / (n * 365)  # Time delta per one step (as fraction of year)
u = 1.30  
d = 1 / u  
rf = 0  
dy = 0  
pu = (1 + rf - dy - d) / (u - d)  
pd = 1 - pu  

assert side in [call, put] and style in [american, european]
print('%18s : %0.8f' % ('Node prob U', pu))
print('%18s : %0.8f' % ('Node prob D', pd))
print('%18s : %0.8f' % ('Node tdelta', tdelta))
print('%18s : %0.8f' % ('Node discount f', rf))

       Node prob U : 0.43478261
       Node prob D : 0.56521739
       Node tdelta : 0.20000000
   Node discount f : 0.00000000


In [5]:
print('Binomial Tree')

# Generate terminal nodes of binomial tree
level = []
print()
for j in range(0, n + 1):
    pr = price * d ** j * u ** (n - j)
    # Option value at the node (depending on side)
    ov = max(0.0, pr - strike) if side == call else max(0.0, strike - pr)
    level.append((pr, ov))
    print('Node [%i,%i] \t Price %6.3f \t Option Value %6.3f' % (n, j, pr, ov))

levels = [None, None, None]

# reduce binomial tree
for i in range(n - 1, -1, -1):  # [n-1 to 0]
    levelNext = []
    print()
    for j in range(0, i + 1):  
        node_u, node_d = level[j], level[j + 1]
        # Instrument's price at the node
        pr = node_d[0] / d
        # Option value at the node (depending on side)
        ov = (node_d[1] * pd + node_u[1] * pu) / (1 + rf)
        if style == american:  # for American options
            ov = max(ov, pr - strike if side == call else strike - pr)
        levelNext.append((pr, ov))
        print('Node [%i,%i] \t Price %6.3f \t Option Value %6.3f' % (i, j, pr, ov))
    level = levelNext
    if j <= 2: levels[j] = level 

Binomial Tree

Node [5,0] 	 Price 352.728 	 Option Value 262.728
Node [5,1] 	 Price 208.715 	 Option Value 118.715
Node [5,2] 	 Price 123.500 	 Option Value 33.500
Node [5,3] 	 Price 73.077 	 Option Value  0.000
Node [5,4] 	 Price 43.241 	 Option Value  0.000
Node [5,5] 	 Price 25.586 	 Option Value  0.000

Node [4,0] 	 Price 271.330 	 Option Value 181.330
Node [4,1] 	 Price 160.550 	 Option Value 70.550
Node [4,2] 	 Price 95.000 	 Option Value 14.565
Node [4,3] 	 Price 56.213 	 Option Value  0.000
Node [4,4] 	 Price 33.262 	 Option Value  0.000

Node [3,0] 	 Price 208.715 	 Option Value 118.715
Node [3,1] 	 Price 123.500 	 Option Value 38.906
Node [3,2] 	 Price 73.077 	 Option Value  6.333
Node [3,3] 	 Price 43.241 	 Option Value  0.000

Node [2,0] 	 Price 160.550 	 Option Value 73.606
Node [2,1] 	 Price 95.000 	 Option Value 20.495
Node [2,2] 	 Price 56.213 	 Option Value  2.753

Node [1,0] 	 Price 123.500 	 Option Value 43.587
Node [1,1] 	 Price 73.077 	 Option Value 10.467

Node [0

Value of the derivative at each node can be seen in the last column above

Value of the call option at time 0 = 24.86

### <font color=green> American call option: </font> Benefit from early exercise

The value of any node in an American option is governed by the function  <b style="color: green">max(intrinsic value, expected discounted value)</b>

$V_n = \max\left(G_n,\frac{pV_{n +1}H^d + qV_{n + 1}H^u}{1 + r}\right)$   


In the case of American options we will benefit from exercising the option at an 'optimal time'.<br>
    Optimal exercise price: $\min\{n: V_n = G_n\}$

Note that $V_n = G_n$ when $E(V_{n+1})\leq G_n$



In [6]:
print('G(n) Process for American Call')

# Generate terminal nodes of binomial tree
level = []
print()
for j in range(0, n + 1):
    pr = price * d ** j * u ** (n - j)
    # Option value at the node (depending on side)
    ov = max(0.0, pr - strike) if side == call else max(0.0, strike - pr)
    gv = max(0, pr - strike)

    level.append((pr, ov, gv))
    print('Node [%i,%i] \t G(n) %6.3f' % (n, j, gv))

levels = [None, None, None]

# reduce binomial tree
for i in range(n - 1, -1, -1):  # [n-1 to 0]
    levelNext = []
    print()
    for j in range(0, i + 1):  
        node_u, node_d = level[j], level[j + 1]
        # Instrument's price at the node
        pr = node_d[0] / d
        # Option value at the node (depending on side)
        ov = (node_d[1] * pd + node_u[1] * pu) / (1 + rf)
        gv = max(0, pr - strike)

        if style == american:  # for American options
            ov = max(ov, pr - strike if side == call else strike - pr)
        levelNext.append((pr, ov, gv))
        print('Node [%i,%i] \t G(n) %6.3f' % (i, j, gv))
    level = levelNext
    if j <= 2: levels[j] = level 

G(n) Process for American Call

Node [5,0] 	 G(n) 262.728
Node [5,1] 	 G(n) 118.715
Node [5,2] 	 G(n) 33.500
Node [5,3] 	 G(n)  0.000
Node [5,4] 	 G(n)  0.000
Node [5,5] 	 G(n)  0.000

Node [4,0] 	 G(n) 181.330
Node [4,1] 	 G(n) 70.550
Node [4,2] 	 G(n)  5.000
Node [4,3] 	 G(n)  0.000
Node [4,4] 	 G(n)  0.000

Node [3,0] 	 G(n) 118.715
Node [3,1] 	 G(n) 33.500
Node [3,2] 	 G(n)  0.000
Node [3,3] 	 G(n)  0.000

Node [2,0] 	 G(n) 70.550
Node [2,1] 	 G(n)  5.000
Node [2,2] 	 G(n)  0.000

Node [1,0] 	 G(n) 33.500
Node [1,1] 	 G(n)  0.000

Node [0,0] 	 G(n)  5.000


We can see that the $V_n = G_n$ is true for the following nodes: 
* [3,3] ; t = 3
* [3,0] ; t = 3
* [4,1] ; t = 4
<br><br>

The above nodes will the optimal time for early exercise. <br>
The rest of the paths will only be exercised at expiry.

### General case: Early exercise of <font color=green>American Call Option </font>
In our case, we are getting early stopping times for exercising a Call only because the risk free rate r=0. We will not get these early stopping times in a setting where r is greater 0.<br><br>
Generally speaking, it can be proved that there is never an optimal time to exercise an American call if the stock is not dividend paying. If the stock is dividend paying, it might make sense to exercise the call prior to the dividend, depending on the value of the dividend.<br><br>
Another perspective to this problem is that if the owner of an American option does not exercise the option at the ‘optimal time’, a replicating portfolio will be able to "consume" profits by re-hedging at a lower price.

<br> **Note: that the comparison here is between exercising the option vs selling the option in the open market, not between exercising the option and holding it to expiry!**

## <font color=orange>American Put Option<font color=darkblue>

In [7]:
side = put  # Option side
style = american  # Option style
price = 95  #Current instrument price
strike = 90  #Strike price
riskfree = 0  
divyield = 0
tte = 365  # Time to expiration in days

print('Calculation Inputs')
print('%18s : %0.3f' % ('Price', price))
print('%18s : %0.3f' % ('Strike', strike))
print('%18s : %0.3f' % ('Risk-free', riskfree))
print('%18s : %0.3f' % ('Div Yield', divyield))
print('%18s : %0.3f' % ('TTE Days', tte))
print()

Calculation Inputs
             Price : 95.000
            Strike : 90.000
         Risk-free : 0.000
         Div Yield : 0.000
          TTE Days : 365.000



In [8]:
n = 5  # Depth of binomial tree (levels are numbered from 0 to n)
tdelta = tte / (n * 365)  # Time delta per one step (as fraction of year)
u = 1.30  # Up movement per step
d = 1 / u  # Down movement per step
rf = 0  # Risk-free rate per step
dy = 0  # Dividend yield per step
pu = (1 + rf - dy - d) / (u - d)  # Probability of up movement
pd = 1 - pu  # Probability of down movement

assert side in [call, put] and style in [american, european]
print('%18s : %0.8f' % ('Node prob U', pu))
print('%18s : %0.8f' % ('Node prob D', pd))
print('%18s : %0.8f' % ('Node tdelta', tdelta))
print('%18s : %0.8f' % ('Node discount f', rf))

       Node prob U : 0.43478261
       Node prob D : 0.56521739
       Node tdelta : 0.20000000
   Node discount f : 0.00000000


In [9]:
print('Binomial Tree')

# Generate terminal nodes of binomial tree
level = []
print()
for j in range(0, n + 1):
    pr = price * d ** j * u ** (n - j)
    # Option value at the node (depending on side)
    ov = max(0.0, pr - strike) if side == call else max(0.0, strike - pr)
    level.append((pr, ov))
    print('Node [%i,%i] \t Price %6.3f \t Option Value %6.3f' % (n, j, pr, ov))

levels = [None, None, None]

# reduce binomial tree
for i in range(n - 1, -1, -1):  # [n-1 to 0]
    levelNext = []
    print()
    for j in range(0, i + 1):  
        node_u, node_d = level[j], level[j + 1]
        # Instrument's price at the node
        pr = node_d[0] / d
        # Option value at the node (depending on side)
        ov = (node_d[1] * pd + node_u[1] * pu) / (1 + rf)
        if style == american:  # for American options 
            ov = max(ov, pr - strike if side == call else strike - pr)
        levelNext.append((pr, ov))
        print('Node [%i,%i] \t Price %6.3f \t Option Value %6.3f' % (i, j, pr, ov))
    level = levelNext
    if j <= 2: levels[j] = level 

Binomial Tree

Node [5,0] 	 Price 352.728 	 Option Value  0.000
Node [5,1] 	 Price 208.715 	 Option Value  0.000
Node [5,2] 	 Price 123.500 	 Option Value  0.000
Node [5,3] 	 Price 73.077 	 Option Value 16.923
Node [5,4] 	 Price 43.241 	 Option Value 46.759
Node [5,5] 	 Price 25.586 	 Option Value 64.414

Node [4,0] 	 Price 271.330 	 Option Value  0.000
Node [4,1] 	 Price 160.550 	 Option Value  0.000
Node [4,2] 	 Price 95.000 	 Option Value  9.565
Node [4,3] 	 Price 56.213 	 Option Value 33.787
Node [4,4] 	 Price 33.262 	 Option Value 56.738

Node [3,0] 	 Price 208.715 	 Option Value  0.000
Node [3,1] 	 Price 123.500 	 Option Value  5.406
Node [3,2] 	 Price 73.077 	 Option Value 23.256
Node [3,3] 	 Price 43.241 	 Option Value 46.759

Node [2,0] 	 Price 160.550 	 Option Value  3.056
Node [2,1] 	 Price 95.000 	 Option Value 15.495
Node [2,2] 	 Price 56.213 	 Option Value 36.540

Node [1,0] 	 Price 123.500 	 Option Value 10.087
Node [1,1] 	 Price 73.077 	 Option Value 27.390

Node [0,0] 

The price of the put option at time = 0 is 19.867

### <font color=orange>American Put option: </font> Benefits from early exercise

The value of any node in an American option is governed by the function  <b style="color: green">max(intrinsic value, expected discounted value)</b>

$V_n = \max\left(G_n,\frac{pV_{n +1}H^d + qV_{n + 1}H^u}{1 + r}\right)$   


Yes, in the case of American options we will benefit from exercising the option at a minimal optimal exercise time.<br><br>
    Optimal exercise price: $\min\{n: V_n = G_n\}$

Note that $V_n = G_n$ when $E(V_{n+1})\leq G_n$

In [10]:
print('G(n) Process for American Put')

# Generate terminal nodes of binomial tree
level = []
print()
for j in range(0, n + 1):
    pr = price * d ** j * u ** (n - j)
    # Option value at the node (depending on side)
    ov = max(0.0, pr - strike) if side == call else max(0.0, strike - pr)
    gv = max(0, strike - pr)

    level.append((pr, ov, gv))
    print('Node [%i,%i] \t G(n) %6.3f' % (n, j, gv))

levels = [None, None, None]

# reduce binomial tree
for i in range(n - 1, -1, -1):  # [n-1 to 0]
    levelNext = []
    print()
    for j in range(0, i + 1):  
        node_u, node_d = level[j], level[j + 1]
        # Instrument's price at the node
        pr = node_d[0] / d
        # Option value at the node (depending on side)
        ov = (node_d[1] * pd + node_u[1] * pu) / (1 + rf)
        gv = max(0, strike - pr)

        if style == american:  # for American options
            ov = max(ov, pr - strike if side == call else strike - pr)
        levelNext.append((pr, ov, gv))
        print('Node [%i,%i] \t G(n) %6.3f' % (i, j, gv))
    level = levelNext
    if j <= 2: levels[j] = level 

G(n) Process for American Put

Node [5,0] 	 G(n)  0.000
Node [5,1] 	 G(n)  0.000
Node [5,2] 	 G(n)  0.000
Node [5,3] 	 G(n) 16.923
Node [5,4] 	 G(n) 46.759
Node [5,5] 	 G(n) 64.414

Node [4,0] 	 G(n)  0.000
Node [4,1] 	 G(n)  0.000
Node [4,2] 	 G(n)  0.000
Node [4,3] 	 G(n) 33.787
Node [4,4] 	 G(n) 56.738

Node [3,0] 	 G(n)  0.000
Node [3,1] 	 G(n)  0.000
Node [3,2] 	 G(n) 16.923
Node [3,3] 	 G(n) 46.759

Node [2,0] 	 G(n)  0.000
Node [2,1] 	 G(n)  0.000
Node [2,2] 	 G(n) 33.787

Node [1,0] 	 G(n)  0.000
Node [1,1] 	 G(n) 16.923

Node [0,0] 	 G(n)  0.000


We can see that the $V_n = G_n$ is true for the following nodes: 
* [3,3] ; t = 3
* [3,0] ; t = 3
* [4,3] ; t = 4
<br><br>

The above nodes will the optimal time for early exercise. <br>
The rest of the paths will only be exercised at expiry.

### General case: Early exercise of <font color=orange> American Put option </font>

In the case of a Put, we will have optimal early exercise situations even if the risk-free rate r is set to any value greater than 0. <br>
* This is different from the case of a Call option where any r ≠ 0 meant that there was no optimal early exercise time for non-dividend paying Calls. <br>

**Note: that the comparison here is between exercising the option vs selling the option in the open market, not between exercising the option and holding it to expiry!**